# Week 8 Seminar: From Binary Logistic Regression to Multilayer Neural Networks




We finished last week's seminar by building a logistic regression model for a sentiment classification task. The dataset used wass 10000 reviews on Yelp classified as negative (1 or 2 star) or positive (3 or 4 star).

The code for reading in, preprocessing, one-hot encoding and spliting the data is as follows. Please run this one cell now as it will take a minute or two to complete

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

!wget https://raw.githubusercontent.com/cbannard/lela60331_24-25/refs/heads/main/data/yelp_reviews.txt

# Create lists
reviews=[]
labels=[]

with open("yelp_reviews.txt") as f:
   # iterate over the lines in the file
   for line in f.readlines()[1:]:
        # split the current line into a list of two element - the review and the label
        fields = line.rstrip().split('\t')
        # put the current review in the reviews list
        reviews.append(fields[0])
        # put the current sentiment rating in the labels list
        labels.append(fields[1])

from collections import Counter
import re

#### This is where we define the character to split on
token_def = re.compile("[^ ]+")
####

# Tokenise the text, turning a list of strings into a list of lists of tokens. We use very naive space-based tokenisation.
tokenized_sents = [token_def.findall(txt) for txt in reviews]
# Collapse all tokens into a single list
tokens=[]
for s in tokenized_sents:
      tokens.extend(s)
# Count the tokens in the tokens list. The returns a list of tuples of each token and count
counts=Counter(tokens)
# Sort the tuples. The reverse argument instructs to put most frequent first rather than last (which is the default)
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
so=list(zip(*so))[0]
# Select the firs 5000 words in the list
type_list=so[0:5000]

# Create a 10000 x 5000 matrix of zeros
M = np.zeros((len(reviews), len(type_list)))
#iterate over the reviews
for i, rev in enumerate(reviews):
    # Tokenise the current review:
    tokens = token_def.findall(rev)
    # iterate over the words in our type list (the set of 5000 words):
    for j,t in enumerate(type_list):
        # if the current word j occurs in the current review i then set the matrix element at i,j to be one. Otherwise leave as zero.
        if t in tokens:
              M[i,j] = 1

train_ints=np.random.choice(len(reviews),int(len(reviews)*0.8),replace=False)
test_ints=list(set(range(0,len(reviews))) - set(train_ints))
M_train = M[train_ints,]
M_test = M[test_ints,]
labels_train = [labels[i] for i in train_ints]
labels_test = [labels[i] for i in test_ints]


We then built a logistic regression model. The code is below:

In [ ]:
import math

num_features=5000
y=[int(l == "positive") for l in labels_train]
weights = np.random.rand(num_features)
bias=np.random.rand(1)
n_iters = 2500
lr=0.1
logistic_loss=[]
num_samples=len(y)
for i in range(n_iters):
  z = M_train.dot(weights)+bias
  q = 1/(1+np.exp(-z))
  eps=0.00001
  loss = -sum((y*np.log2(q+eps)+(np.ones(len(y))-y)*np.log2(np.ones(len(y))-q+eps)))
  logistic_loss.append(loss)

  dw = ((q-y).dot(M_train) * (1/num_samples))
  db = sum((q-y))/num_samples
  weights = weights - lr*dw
  bias = bias - lr*db

plt.plot(range(1,n_iters),logistic_loss[1:])
plt.xlabel("number of epochs")
plt.ylabel("loss")
#loss = sum(-(np.ones(len(y))*np.log2(q)+(np.ones(len(y))-y)*np.log2(np.ones(len(y))-q)))

Now that we have a fitting model, we can use it to predict labels for our test items. The test reviews are in the one-hot matrix M_test. The labels for the test reviews are in the list labels_test.

Problem 1: Complete the code below so that it calculate the vector of predicted values (0 or 1) y_test_pred for our test items.

In [ ]:
  z = ??
  q = ??
  y_test_pred ??

We can calculate accuracy for the performance of our model on the test items as follows:

In [ ]:
y_test=[int(l == "positive") for l in labels_test]
acc_test=[int(yp == y_test[s]) for s,yp in enumerate(y_test_pred)]
print(sum(acc_test)/len(acc_test))

Remember though that precision is not usually a good measure and so we calculate precision and recall.

Problem 2 : Calculate precision and recall values for the performance of our model on the test data. I have given code for calculating the true positive rate. You will need to calculate the rest of the values from the confusion matrix and then use these numbers to calculate our evaluation metrics.

In [ ]:
labels_test_pred=["positive" if s == 1 else "negative" for s in y_test_pred]

In [ ]:
true_positives=sum([int(yp == "positive" and labels_test[s] == "positive") for s,yp in enumerate(labels_test_pred)])

Problem 3: Calculate precision and recall values for the performance of the model on the training data. How do the numbers differ from those you found for the test set? Why do you think this is?

Inspecting weights

### Inspecting weights
Weights in a one-layer network with one-hot encoded inputs are simple to interpret - words with high weights are those whose presence makes a positive outcome most likely and those with low weights are those whose presence makes a negative outcome most likely.

Our weights are in a numpy array, with each weight corresponding the word at the same position (index) in our type_list.

 We can obtain a list of the indices in order to the weight at each position using the numpy function argsort. This sorts from low to high so we take the top 10 to find the indices of the ten words most strongly associated with a negative outcome. Or we can reverse it to find the indices of the ten words most strongly associated with a positive outcome.

 We can ten just look at the words that occur at these indices:

In [ ]:
[type_list[x] for x in np.argsort(weights)[0:20]]

In [ ]:
[type_list[x] for x in np.argsort(weights)[::-1][0:20]]

Problem 4a: Inspect the list of tokens with highest and lowest weights, going beyond the top and bottom 10. Do you spot any that indicate overfitting?

Problem 4b: Look back up to the first code block above where we create the one-hot encoded data. Can you change the way the text is tokenised in order to avoid the problematic tokens you noted in 4a? Retrain the model. How do your changes affect performance / the token lists?


### Exclusive OR problem

Problem 5: Create the data for the AND, OR, and XOR functions. Fit a logistic regression to these problem using the code you have developed in previous problems and then inspect the output. What do you see?

In [ ]:
x=np.array([])
y=np.array([])

### Your first multilayer network

Problem 6: Enter the weights from the example multilayer network from the lecture and demonstrate that it can solve the XOR problem
Remember that you have two sets of weights - those from layer 0 to layer 1 and those from layer 1 to layer 2 - and that the former is a matrix not a vector.
